In [ ]:
# Доверительный интервал на std, bootstrap, ЦПТ

In [14]:
from scipy.stats import t, norm
import numpy as np
from scipy.stats import bootstrap

In [2]:
data = np.random.poisson(lam=50, size=1000)

In [27]:
bootstrap(
    data=(data,),
    statistic=np.std,
    confidence_level=0.95,
    n_resamples=1000,
    method='percentile',
).confidence_interval

ConfidenceInterval(low=6.691271799258252, high=7.333239563363699)

In [37]:
stds = []
n_iter = 1000
for _ in range(n_iter):
    samples = np.random.poisson(lam=50, size=100)
    std_ = np.std(samples)
    stds.append(std_)

In [38]:
stds = np.array(stds)
stds.mean(), stds.std()

(7.021143603701341, 0.493408042297394)

In [39]:
alpha = 0.05
norm().ppf(q=1-alpha/2)

1.959963984540054

In [40]:
stds = np.array(stds)
mean = stds.mean()
std = stds.std()

alpha = 0.05
z_alpha2 = norm().ppf(q=1-alpha/2)
N = 1000

mean - z_alpha2 * std / N ** 0.5, mean + z_alpha2 * std / N ** 0.5

(6.990562418349836, 7.051724789052845)

А теперь давайте воспользуемся доверительными интервалами, чтобы сделать более точные выводы о метриках модели.

Для начала давайте скачаем данные:

from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)


Это задача предсказания средней цены в области на дом. Давайте оценим 95-процентный доверительный интервал на MSE Ridge-регрессии для этой выборки. Будем использовать бутстрэп для этого.

Для этого в цикле будем генерировать бутстрэп-выборки для обучения модели (то есть брать объекты из выборки с возвращением). В качестве тестовых объектов будем брать оставшиеся объекты. Каждый раз будем обучать модель и оценивать качество.

Реализуйте такой код для 95-процентного доверительного интервала на MSE.

Используйте 1000 итераций генераций выборки и обучения модели. Данные не обрабатывайте. Полезными методами будут np.random.choice для генерации индексов бутстрэп-выборок (не забудьте про «генерацию с возвращением»), np.setdiff1d для получения остатка выборки.

In [42]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [47]:
len(y)

20640

In [75]:
a = np.random.choice(range(len(y)), size=len(y), replace=True)
sum(a), len(np.unique(a)), len(a)

(213276099, 13022, 20640)

In [43]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

n_iterations = 1000
alpha = 1.0

mse_values = []

for _ in range(n_iterations):

    bootstrap_indices = np.random.choice(range(len(y)), size=len(y), replace=True)
    X_bootstrap = X.iloc[bootstrap_indices]
    y_bootstrap = y.iloc[bootstrap_indices]

    test_indices = np.setdiff1d(range(len(y)), bootstrap_indices)
    X_test = X.iloc[test_indices]
    y_test = y.iloc[test_indices]

    model = Ridge(alpha=alpha)
    model.fit(X_bootstrap, y_bootstrap)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

In [44]:
data = np.array(mse_values)
bootstrap(
    data=(data,),
    statistic=np.std,
    confidence_level=0.95,
    n_resamples=1000,
    method='percentile',
).confidence_interval

ConfidenceInterval(low=0.6440489885051739, high=1.3486414471900465)

Давайте оценим, насколько хорошие интервалы у нас получается строить.

Напишите функцию, которая генерирует случайную выборку из нормального распределения (давайте возьмём среднее значение, равное 3, среднеквадратичное отклонение, равное 4, а в выборке будет 1000 объектов).

Возьмите из предыдущих заданий код для построения соответствующего 95-процентного доверительного интервала для среднего на основе центральной предельной теоремы.

Повторите процедуру генерации выборки, построения доверительного интервала 1000 раз. Каждый раз проверяйте, действительно ли доверительный интервал покрыл ваше истинное среднее значение. 

Посчитайте долю случаев, когда интервал покрыл истинное значение. Ближе к какому значению вы получили ответ?



In [76]:
p_mean = 3
p_scale = 4
alpha = 0.05
data = np.random.normal(loc=p_mean, scale=p_scale, size=1000)
N = len(data)
mean = np.mean(data)
z_alpha2 = norm().ppf(q=1-alpha/2)
std = np.std(data)

mean - z_alpha2 * std / N ** 0.5, mean + z_alpha2 * std / N ** 0.5

(2.77954541023751, 3.2639939757152585)

In [77]:
def eval_confidence_interval():
    p_mean = 3
    p_scale = 4
    alpha = 0.05
    data = np.random.normal(loc=p_mean, scale=p_scale, size=1000)
    N = len(data)
    mean = np.mean(data)
    z_alpha2 = norm().ppf(q=1-alpha/2)
    std = np.std(data)
    
    if p_mean >= mean - z_alpha2 * std / N ** 0.5 and p_mean <= mean + z_alpha2 * std / N ** 0.5:
        return 1
    else:
        return 0

In [82]:
sum_ = 0
for i in range(1000):
    sum_ += eval_confidence_interval()

sum_

959